In [1]:
%%writefile data.csv
Fund,P1,P2,P3,P4
M1,0.67,0.45,6.5,42.6
M2,0.60,0.36,3.6,53.3
M3,0.82,0.67,3.8,63.1
M4,0.60,0.36,3.5,69.2
M5,0.76,0.58,4.8,43.0
M6,0.69,0.48,6.6,48.7
M7,0.79,0.62,4.8,59.2
M8,0.84,0.71,6.5,34.5


Writing data.csv


In [2]:
%%writefile topsis.py
import sys
import pandas as pd
import numpy as np

def main():
    if len(sys.argv) != 5:
        print("Usage: python topsis.py <InputDataFile> <Weights> <Impacts> <OutputFileName>")
        sys.exit(1)

    input_file = sys.argv[1]
    weights = sys.argv[2]
    impacts = sys.argv[3]
    output_file = sys.argv[4]

    try:
        data = pd.read_csv(input_file)
    except FileNotFoundError:
        print("Error: File not found.")
        sys.exit(1)

    if data.shape[1] < 3:
        print("Error: Input file must contain at least 3 columns.")
        sys.exit(1)

    numeric_data = data.iloc[:, 1:]

    try:
        numeric_data = numeric_data.astype(float)
    except:
        print("Error: From 2nd to last column, all values must be numeric.")
        sys.exit(1)

    weights = weights.split(",")
    impacts = impacts.split(",")

    if len(weights) != numeric_data.shape[1] or len(impacts) != numeric_data.shape[1]:
        print("Error: Number of weights, impacts, and columns must be same.")
        sys.exit(1)

    try:
        weights = np.array(weights, dtype=float)
    except:
        print("Error: Weights must be numeric.")
        sys.exit(1)

    for i in impacts:
        if i not in ['+', '-']:
            print("Error: Impacts must be either + or -.")
            sys.exit(1)

    norm_data = numeric_data / np.sqrt((numeric_data**2).sum())

    weighted_data = norm_data * weights

    ideal_best = []
    ideal_worst = []

    for i in range(len(impacts)):
        if impacts[i] == '+':
            ideal_best.append(weighted_data.iloc[:, i].max())
            ideal_worst.append(weighted_data.iloc[:, i].min())
        else:
            ideal_best.append(weighted_data.iloc[:, i].min())
            ideal_worst.append(weighted_data.iloc[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)

    dist_best = np.sqrt(((weighted_data - ideal_best) ** 2).sum(axis=1))
    dist_worst = np.sqrt(((weighted_data - ideal_worst) ** 2).sum(axis=1))

    score = dist_worst / (dist_best + dist_worst)

    data["Topsis Score"] = score
    data["Rank"] = data["Topsis Score"].rank(ascending=False).astype(int)

    data.to_csv(output_file, index=False)

    print("TOPSIS completed successfully!")
    print(f"Output saved to {output_file}")

if __name__ == "__main__":
    main()


Writing topsis.py


In [3]:
!python topsis.py data.csv "1,1,1,1" "+,+,+,+" output.csv

TOPSIS completed successfully!
Output saved to output.csv


In [4]:
import pandas as pd
pd.read_csv("output.csv")


,Fund,P1,P2,P3,P4,Topsis Score,Rank
0,M1,0.67,0.45,6.5,42.6,0.462285,5
1,M2,0.60,0.36,3.6,53.3,0.267624,8
2,M3,0.82,0.67,3.8,63.1,0.601510,2
3,M4,0.60,0.36,3.5,69.2,0.411355,7
4,M5,0.76,0.58,4.8,43.0,0.453830,6
5,M6,0.69,0.48,6.6,48.7,0.535870,4
6,M7,0.79,0.62,4.8,59.2,0.636224,1
7,M8,0.84,0.71,6.5,34.5,0.585365,3


In [5]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful